# VacationPy
----

#### Note
* api_keys file was outside of VacationPy dir. i.e at same level sa WeatherPy & VacationPy dir hence need to append path in order to import keys


* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [12]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint
# Import API key
import sys
sys.path.append('..')
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [13]:
# Store the csv file path in file_path variable
file_path = "../WeatherPy/city_weather_data.csv"

# Read csv file & stoer it as DataFrame
city_weather_data = pd.read_csv(file_path, encoding="utf-8")

# Display DataFrame
city_weather_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,nizhniy odes,63.64,54.86,37.78,96.0,100.0,13.04,RU,1.603910e+09
1,vaini,-21.20,-175.20,69.80,83.0,20.0,8.05,TO,1.603910e+09
2,clyde river,70.47,-68.59,15.80,78.0,75.0,16.11,CA,1.603910e+09
3,rikitea,-23.12,-134.97,74.75,83.0,11.0,13.91,PF,1.603910e+09
4,khatanga,71.97,102.50,-1.70,87.0,0.0,4.45,RU,1.603910e+09
...,...,...,...,...,...,...,...,...,...
599,ucluelet,48.93,-125.55,54.00,100.0,90.0,5.82,CA,1.603910e+09
600,along,28.17,94.77,61.77,64.0,0.0,2.24,IN,1.603910e+09
601,haicheng,40.85,122.75,39.78,63.0,24.0,2.28,CN,1.603910e+09
602,mantua,45.17,10.78,53.01,91.0,0.0,1.01,IT,1.603910e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [14]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = city_weather_data[["Lat", "Lng"]]

# convert humidity to float and store it in humidity variable
humidity = city_weather_data["Humidity"].astype(float)

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=70,
                                 point_radius=3)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [15]:
# Etracting the rows for cities with matches specified criteria and store it in new DataFrame
ideal_weather_cities = city_weather_data.loc[(city_weather_data["Max Temp"]>70) & 
                                             (city_weather_data["Max Temp"]<75) &
                                             (city_weather_data["Wind Speed"]<10) & 
                                             (city_weather_data["Cloudiness"]==0),:]
# Display count
ideal_weather_cities.count()

City          10
Lat           10
Lng           10
Max Temp      10
Humidity      10
Cloudiness    10
Wind Speed    10
Country       10
Date          10
dtype: int64

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [23]:
# Extract only required columns from ideal_weather_cities to new DataFrame called hotel_df
hotel_df = ideal_weather_cities[["City", "Lat", "Lng", "Country"]].copy()

# Add 3 empty columns to Hotel_df which would hold hotel name & its Lat-Lng
hotel_df["Hotel"] = ''
hotel_df["Hotel_Lat"] = ''
hotel_df["Hotel_Lng"] = ''

# Set the target_search & radius 
target_search = "hotel"
target_radius = 5000

# Define base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Iterate through each row in hotel_df
for index, row in hotel_df.iterrows():
    # Get the Lat, Lng  
    Lat = hotel_df["Lat"][index]
    Lng = hotel_df["Lng"][index]
    city = hotel_df["City"][index]
    coordinates = f"{Lat},{Lng}"
    
    # Define parameters 
    params = {"location" : coordinates,
              "keyword" : target_search,
              "radius" : target_radius,
              "key" : g_key
              }
    
    # Send get request and store data in jason format
    response = requests.get(base_url, params=params).json()
    
    try:
        # Get the name, Lat & Lng of the hotel & store it in respective column 
        print(f"Processing City | {city}")
        hotel_df.loc[index,'Hotel'] = response['results'][0]['name']
        hotel_df.loc[index,'Hotel_Lat'] = response['results'][0]['geometry']['location']['lat']
        hotel_df.loc[index,'Hotel_Lng'] = response['results'][0]['geometry']['location']['lng']
            
    except(KeyError, IndexError):
        # If hotel is not found print the message
        print(f"No Hotel found for city {city}")        

Processing City | tura
Processing City | mareeba
Processing City | margherita
Processing City | guerrero negro
Processing City | cap malheureux
Processing City | barsi
Processing City | jamkhed
Processing City | san rafael
Processing City | sur
Processing City | karratha


In [24]:
# Check if there are any missing values (i.e if Hotel is not found for any city)
hotel_df.count()

City         10
Lat          10
Lng          10
Country      10
Hotel        10
Hotel_Lat    10
Hotel_Lng    10
dtype: int64

In [25]:
# Display Data Frame
hotel_df

,City,Lat,Lng,Country,Hotel,Hotel_Lat,Hotel_Lng
91,tura,25.52,90.22,IN,Hotel Polo Orchid,25.5145,90.1922
129,mareeba,-17.00,145.43,AU,The Cape Gateway Motel,-16.9729,145.416
217,margherita,27.28,95.68,IN,Hotel Royal Treat,27.2887,95.6996
227,guerrero negro,27.98,-114.06,MX,The Halfway Inn,28.001,-114.014
254,cap malheureux,-19.98,57.61,MU,Paradise Cove Boutique Hotel,-19.9879,57.6327
266,barsi,18.23,75.70,IN,Shiv Darshan Lodge,18.2371,75.674
374,jamkhed,18.74,75.31,IN,SAHARA HOTEL,18.7413,75.3
461,san rafael,-34.62,-68.33,AR,Tower Inn Hotel & Suites Casino Spa Convention...,-34.6128,-68.3394
554,sur,22.57,59.53,OM,Sur Plaza Hotel,22.5664,59.4959
593,karratha,-20.74,116.85,AU,ibis Styles Karratha,-20.7322,116.847


In [26]:
# Store only City , Country and Hotel coulmn to new dataframe
new_hotel_df = hotel_df[["City", "Country", "Hotel"]]

# Display Data Frame
new_hotel_df

,City,Country,Hotel
91,tura,IN,Hotel Polo Orchid
129,mareeba,AU,The Cape Gateway Motel
217,margherita,IN,Hotel Royal Treat
227,guerrero negro,MX,The Halfway Inn
254,cap malheureux,MU,Paradise Cove Boutique Hotel
266,barsi,IN,Shiv Darshan Lodge
374,jamkhed,IN,SAHARA HOTEL
461,san rafael,AR,Tower Inn Hotel & Suites Casino Spa Convention...
554,sur,OM,Sur Plaza Hotel
593,karratha,AU,ibis Styles Karratha


In [28]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in new_hotel_df.iterrows()]
locations = hotel_df[["Hotel_Lat", "Hotel_Lng"]]

In [29]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))